### Regularising Measurements  

Hi Folks, 

### Problem

I've been trying to solve a problem where I have _measurements_ that I want to regularise. 

I have some prior knowledge that 

1. The mean of the measurements is should be around $\mu_0$ 
2. The measurements should be "close" to each other i.e. have low variance. 

### Potential Solution
The solution I have is to model the data as being iid with a Gaussian likelihood and parameterised by a mean $\mu$ and precision, $\tau = 1/\sigma^2$

$$ 
\begin{align}
x_i | \mu, \tau {} & \overset{iid}{\sim}  \mathcal{N}(\mu, \tau)  \\
\mu {} & \sim \mathcal{N}(\mu_0, n_0\tau) \\
\tau {} & \sim \textsf{Gamma}(\alpha, \beta)
\end{align}
$$

Starting with parameters for $\mu_0$, $n_0$, $\alpha$, $\beta$, the posterior mean and precision can be calculated as 
$$
\begin{align}
\mu | \tau, x \sim {} & \mathcal{N}\left( \frac{n}{n+n_0}\bar{x} +  \frac{n_0}{n+n_0}\mu_0 , (n +n_0)\tau   \right) \\
\tau | x \sim  {} & \textsf{Ga}\left( \alpha + \frac{n}{2}, \beta + \frac{1}{2}\sum_i (x_i - \bar{x})^2 + \frac{nn_0}{2(n + n_0)}  (\bar{x} - \mu_0)^2  \right) 
\end{align}
$$
where $\bar{x}$ is the sample mean of the data. [1]

By setting $\mu_0$ to be equal to the prior knowledge of the mean and $n_0$, $\alpha$, $\beta$ arbirtrarily so that the variance of posterior is low, I, can compute the the posterior mean and precision. 


My solution is then to _adjust_ the data such that it matches the posterior mean and precision. 


1. Step 1: Find the $z$ scores with the mean and precision calculated from the data: 
    * $z^{\text{old}}_i  = \sqrt{\tau_x}(x_i - \bar{x})$
where $\tau_x$ is the sample precision. 


2. Step 2: Convert the $z$ scores back but with the posterior mean and precision 
    * $x^{\text{new}}_i  = \frac{1}{\sqrt{\tau}}z^{\text{old}}_i + \mu$

Is this valid? 

Is there a better way to do this?


### References 

[1] [The Conjugate Prior for the Normal Distribution](https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/lectures/lecture5.pdf), Stat260 UC Berkeley


In [53]:
import pandas as pd
import numpy as np
import pymc3 as pm
from typing import List, Dict

<IPython.core.display.Javascript object>

In [43]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [37]:
mu0 = 3

n0 = 1
alpha = 10
beta = 1

tau_mean_prior = alpha/beta

print(f"Prior mean, mu0 {mu0}", f"Prior precision, tau0 {tau_mean_prior: .2f}")

x = np.array([1, 2, 4, 6])

n = len(x)
x_bar = np.mean(x)
tau_x = 1/np.var(x)

print("")

print(f"Data mean, x_bar {x_bar}", f"Data precision, tau_x {tau_x: .2f}")


alpha_post = alpha + n/2
beta_post = beta + 0.5*np.sum( (x - x_bar)**2) + 0.5*n*n0/(n + n0)*(x_bar - mu0)**2
tau_mean_post = alpha_post/beta_post


mu_mean_post = (n/(n + n0))*x_bar + (n0/(n + n0))*mu0

print("---")
print(f"Posterior mean, {mu_mean_post: .2f}", f"Posterior precision, {tau_mean_post: .2f}")


Prior mean, mu0 3 Prior precision, tau0  10.00

Data mean, x_bar 3.25 Data precision, tau_x  0.27
---
Posterior mean,  3.20 Posterior precision,  1.43


In [23]:
x = np.array([1, 2, 4, 6])

with pm.Model() as mod:
    # Priors
    mu0 = 3
    n0 = 1
    alpha = 10
    beta = 1
         
    tau = pm.Gamma("tau", alpha=alpha, beta=beta)
    mu = pm.Normal("mu", mu=mu0, tau=n0*tau)
    
    # Likelihood 
    y = pm.Normal("y", mu=mu, tau=tau, observed=x)
    
    # Magic     
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, tau]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 19 seconds.


In [24]:
mu_sample_post = np.mean(trace.get_values("mu"))
tau_sample_post = np.mean(trace.get_values("tau"))
print("NUTS sampler results")
print(f"Posterior mean, {mu_sample_post: .2f}", f"Posterior precision, {tau_sample_post: .2f}")

NUTS sampler results
Posterior mean,  4.53 Posterior precision,  0.15


In [38]:
z_scores = np.sqrt(tau_x)*(x - x_bar)
z_scores

x_transformed = (1/np.sqrt(tau_mean_post))*z_scores + mu_mean_post
x_transformed

array([2.21968528, 2.65538071, 3.52677157, 4.39816243])

In [91]:
df = pd.DataFrame(
    {"hierarchy": ["A"] * 4 + ["B"] * 5, "val": [1, 2, 4, 6] + [-1, -2, -4, -6, 0]}
).assign(**{"item": lambda x: x["hierarchy"] + x.index.astype("str")})


priors = {
    "A": {"mu0": 3, "n0": 1, "alpha": 10, "beta": 1,},
    "B": {"mu0": -3, "n0": 1, "alpha": 10, "beta": 1,},
}

df_prior = pd.DataFrame(priors).T.reset_index().rename(columns={"index": "hierarchy"})

display(df_prior)


def calc_sample_stats(df: pd.DataFrame, agg_lvl: List[str]) -> pd.DataFrame:
    """Regularises by hierarchy"""
    df_sample_stats = (
        df.groupby(agg_lvl)
        .agg(
            **{
                "sample_mean": ("val", np.mean),
                "sample_var": ("val", np.var),
                "n": ("val", "count"),
            }
        )
        .assign(**{"sample_precision": lambda x: 1 / x["sample_var"]})
    ).reset_index()
    return df_sample_stats


toto = calc_sample_stats(df, agg_lvl=["hierarchy"])


def update_posterior(
    df_sample_stats: pd.DataFrame, df_prior: pd.DataFrame, agg_lvl: List[str]
) -> pd.DataFrame:
    """Update posterior distribution parameters"""
    df_with_prior = df_sample_stats.merge(
        df_prior, on=agg_lvl, how="left", validate="m:1"
    )
    df_with_prior = df_with_prior.assign(
        **{
            "posterior_mean": lambda x: (x["n"] / (x["n"] + x["n0"])) * x["sample_mean"]
            + (x["n0"] / (x["n"] + x["n0"])) * x["mu0"],
            "posterior_alpha": lambda x: x["alpha"] + x["n"] / 2,
            "posterior_beta": lambda x: x["beta"]
            + (0.5 * x["sample_var"] * x["n"])
            + (0.5 * x["n"] * x["n0"] / (x["n"] * x["n0"]))
            * (x["sample_mean"] - x["mu0"]) ** 2,
            "posterior_precision": lambda x: x["posterior_alpha"] / x["posterior_beta"],
        }
    )

    return df_with_prior


lala = update_posterior(toto, df_prior, agg_lvl=["hierarchy"])

cols = [
    "hierarchy", 
    "sample_mean", 
    "sample_precision", 
    "posterior_mean", 
    "posterior_precision" 
]

lala = lala[cols]

lala



,hierarchy,mu0,n0,alpha,beta
0,A,3,1,10,1
1,B,-3,1,10,1


,hierarchy,sample_mean,sample_precision,posterior_mean,posterior_precision
0,A,3.25,0.203390,3.200000,1.104506
1,B,-2.60,0.172414,-2.666667,0.802311


<IPython.core.display.Javascript object>

In [94]:
(
    df.merge(lala, on=["hierarchy"], how="left", validate="m:1").assign(
        **{
            "z_old": lambda x: np.sqrt(x["sample_precision"])
            * (x["val"] - x["sample_mean"]),
            "val_new": lambda x: x["z_old"] / np.sqrt(x["posterior_precision"])
            + x["posterior_mean"],
        }
    )
)

,hierarchy,val,item,sample_mean,sample_precision,posterior_mean,posterior_precision,z_old,val_new
0,A,1,A0,3.25,0.203390,3.200000,1.104506,-1.014722,2.234476
1,A,2,A1,3.25,0.203390,3.200000,1.104506,-0.563735,2.663598
2,A,4,A2,3.25,0.203390,3.200000,1.104506,0.338241,3.521841
3,A,6,A3,3.25,0.203390,3.200000,1.104506,1.240216,4.380085
4,B,-1,B4,-2.60,0.172414,-2.666667,0.802311,0.664364,-1.924956
5,B,-2,B5,-2.60,0.172414,-2.666667,0.802311,0.249136,-2.388525
6,B,-4,B6,-2.60,0.172414,-2.666667,0.802311,-0.581318,-3.315664
7,B,-6,B7,-2.60,0.172414,-2.666667,0.802311,-1.411773,-4.242802
8,B,0,B8,-2.60,0.172414,-2.666667,0.802311,1.079591,-1.461386


<IPython.core.display.Javascript object>

In [77]:
x = np.array([1, 2, 4, 6])

n = len(x)
x_bar = np.mean(x)
tau_x = 1 / np.var(x)
(np.sum((x - x_bar) ** 2)), (np.var(x) * (n))

(14.75, 14.75)

<IPython.core.display.Javascript object>